In [1]:
import gym
import random
#from torch import *
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy

In [2]:
from game_main import *

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
env = BestGameEver()

In [4]:
#env = gym.make('CartPole-v1')

In [5]:
#env.spec.reward_threshold

In [6]:
class RLModel(nn.Module):
    def __init__(self):
        super(RLModel, self).__init__()
        self.linear1 = nn.Linear(34, 128)
        self.drop1 = nn.Dropout(p=.4)
        self.linear2 = nn.Linear(128, 4)
        
        self.old_params = self.state_dict()
    
    def save(self):
        self.old_params = deepcopy(self.state_dict())
    
    def old_forward(self, x):
        current_params = deepcopy(self.state_dict())
        self.load_state_dict(self.old_params)
        #print('old', self.state_dict()['linear1.weight'][0])
        out = self.forward(x)
        self.load_state_dict(current_params)
        #print('current', self.state_dict()['linear1.weight'][0])
        return out
        
    def forward(self, x):
        #print('current', self.state_dict()['linear1.weight'][0])
        x = self.linear1(x)
        x = self.drop1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.softmax(x, dim=-1)
        
        return x

In [7]:
gamma = .99

In [8]:
def one_episode(my_model, speed=False):
    observation = np.array(env.reset())
    print(speed)

    loss = []
    old_loss = []
    rewards = []

    for i in range(1000):
        probs = my_model(torch.from_numpy(observation).float())
        probs_old = my_model.old_forward(torch.from_numpy(observation).float())
        #env.render()
        #env.action_space.sample()
        
        np_probs = probs.detach().numpy()
        np_probs_old = probs_old.detach().numpy()
        
        action = np.random.choice([0, 1, 2, 3], 1, p=(np_probs/sum(np_probs)))[0]
        loss.append(torch.log(probs[action]))
        old_loss.append(torch.log(probs_old[action]))
        
        #print(probs, action, old_loss[-1])
        #print(old_loss)
        #action = random.choice([0, 1])
        #observation, reward, done, info = env.step(action) # take a random action
        observation, reward, done = env.step(action, speed)
        #print(observation, reward, done)
        observation = np.array(observation)
        rewards.append(reward)
        #print(observation, done, reward, info)

        if done:
            break
            
    for i in range(len(rewards)-1, -1, -1):
        rewards[i] = rewards[i] + (rewards[i+1] * gamma) if i != len(rewards)-1 else 1
    
    return rewards, loss, old_loss

In [9]:
def train_loop():
    
    my_model = RLModel()
    optimizer = torch.optim.Adam(my_model.parameters(), lr=0.01)
    
    running_mean = 0
    
    for i in range(100000):
        rewards, loss, old_loss = one_episode(my_model, speed=abs(running_mean - 1) < .1)
        
        #print(loss, old_loss)
        
        running_mean = running_mean * .99 + len(rewards) * .01
        
        if i%100 == 0:
            print(running_mean)
        
        
        loss = torch.stack(loss)
        old_loss = torch.stack(old_loss).detach()

        rewards = np.array(rewards)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 0.00001)
        
        rewards = torch.from_numpy(rewards).float()
        
        #print(rewards)

        #my_model.save()

        #optimizer.zero_grad()
        #reduced_loss = -1 * torch.mean((loss * rewards))
        #reduced_loss = -1 * torch.mean(torch.exp(loss - old_loss) * rewards)
        
        
        #reduced_old_loss = -1 * torch.mean((old_loss * rewards))
        
        #print(reduced_loss)
        #reduced_loss.backward()
        #optimizer.step()
        #continue
        
        my_model.save()
        
        epsilon = .2
        
        optimizer.zero_grad()
        #ratio = torch.ones_like(loss)
        ratio = torch.exp(loss - old_loss)
        
        #print(ratio)
        #print(ratio)
        surr1 = ratio * rewards
        surr2 = torch.clamp(ratio, 1 - epsilon, 1 + epsilon) * rewards
        #print(surr1, surr2)
        
        policy_surr = -1 * torch.min(surr1, surr2).mean()
        #policy_surr = -1 * surr1.mean()
        
        torch.nn.utils.clip_grad_norm_(my_model.parameters(), 30)
        
        policy_surr.backward()
        #reduced_loss.backward()
        optimizer.step()

In [10]:
train_loop()

False
We win, motherfucker
1.82
False
We win, motherfucker
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


KeyboardInterrupt: 